<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/tidy_up_preprocessing_notebook/notebooks/processed/ct_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [398]:
"""
===================================================
Author: Chiaki Tachikawa
Role: Data Science Lead, Bank of England Employer Project (Quant Collective)
LinkedIn: https://www.linkedin.com/in/chiaki-tachikawa
Date: 2025-02-13
Version: 1.1

Description:
    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:
    - Importing necessary libraries and downloading NLTK data.
    - Defining and applying a `preprocessor` function to clean and tokenize text data.
    - Reading and preprocessing various CSV files containing transcript data.
    - Exporting the preprocessed data to new CSV files for further analysis.

===================================================
"""

'\n===================================================\nAuthor: Chiaki Tachikawa\nRole: Data Science Lead, Bank of England Employer Project (Quant Collective)\nLinkedIn: https://www.linkedin.com/in/chiaki-tachikawa\nDate: 2025-02-13\nVersion: 1.1\n\nDescription:\n    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:\n    - Importing necessary libraries and downloading NLTK data.\n    - Defining and applying a `preprocessor` function to clean and tokenize text data.\n    - Reading and preprocessing various CSV files containing transcript data.\n    - Exporting the preprocessed data to new CSV files for further analysis.\n\n===================================================\n'

# **Library**

In [399]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download('wordnet')
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from google.colab import drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# **Function**

preprocessor function : The function modifies the DataFrame data in place, adding two new columns (col1 and col2) with preprocessed text.


Input:
  - name of dataframe
  - name of column which contains the text to clean
  - name of column which is tokenized
  - name of column which is cleaned

In [400]:
#create function to preprocess data
def preprocessor (data, col, col1,col2):
  #Copy col1umn
  data[col1]=data[col]
  data[col2]=data[col]


  #Adding column1
  #Lower the lettercase
  data[col1] = data[col1].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col1] = data[col1].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #Tokenize the word
  data[col1] = data[col1].apply(nltk.word_tokenize)

  #Remove numbers
  data[col1] = data[col1].apply(lambda x: [word for word in x if not word.isdigit()])

  #remove symbol from comments
  data[col1] = data[col1].apply(lambda x: [word for word in x if x!=""])

  #remove short word
  data[col1] = data[col1].apply(lambda x: [word for word in x if len(word)>2])

  #remove symbols
  data[col1] = data[col1].apply (lambda x: [re.sub(r"[^a-z]", "", word) for word in x])

  #lemmatization
  lemmatizer = WordNetLemmatizer()
  data[col1] = data[col1].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])



  #Adding column2
  #Lower the lettercase
  data[col2] = data[col2].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col2] = data[col2].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #remove symbols
  data[col2] = data[col2].apply (lambda x: [re.sub(r"[.,'?]", "", x)])

  return


find_row: This function searches upwards from the given current_row_num in the DataFrame df to find the first row where the value in column "M" is "A". It returns the index of that row. If no such row is found, it returns 0

In [401]:
def find_row (df, col, current_row_num):
  #list_name=[]
  i = current_row_num-1
  while i > 0:
    if df[col][i] == "A":
      break
    else:
      i-=1
  return i

find_row_empty: This function searches upwards from the given current_row_num to find the first row where col1 has the value "A" and col2 is not an empty list. It returns the index of that row.

In [402]:
def find_row_empty (df, col1, col2, current_row_num):
  #list_name=[]
  i = current_row_num-1
  while i > 0:
    if df[col1][i] == "A" and df[col2][i] != []:
      break
    else:
      i-=1
  return i

In [403]:
def create_ques_num_column (data, new_col,marker_col):
  #Create question number column
  data[new_col]=None
  #set global var to count question number
  num = 0
  #if Q was found, num adds 1 otherwise none
  for i in data.index:
    if data.loc[i,marker_col]=="Q":
      data.at[i,new_col]=num
      num +=1
    else:
      continue

In [404]:
# Function to extract names
def extract_name(full_string):
    return full_string.split(',')[0]

In [405]:
#check if there is "A" before "Q" from the current location
def find_last_a (df, col, current_row_num):
  #list_name=[]
  i = current_row_num-1
  while i > 0:
    if df[col][i] == "Q":
      j = i-1
      while j > 0:
        if df[col][j]=="A":
          pass
        else:
          break
        j-=1
      break
    else:
      i-=1
  return i

## **Data**

In [406]:
#drive.mount('/content/drive')

In [407]:
#!ls"/content/bank_of_england/data/preprocessed_data/Archived/jpmorgan_qa_section_preprocessed.csv"

In [408]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


JP Morgan QA section

In [409]:
"""
#Obtaining management discussion / git bash
!git clone https://github.com/sheldonkemper/bank_of_england.git
!git switch Preprocessing
%cd bank_of_england/data/preprocessed_data/archived
%ls
"""

'\n#Obtaining management discussion / git bash\n!git clone https://github.com/sheldonkemper/bank_of_england.git\n!git switch Preprocessing\n%cd bank_of_england/data/preprocessed_data/archived\n%ls\n'

In [410]:
#Defining qa_data
qa_data = pd.read_csv("/content/drive/MyDrive/bank_of_england/data/preprocessed_data/original_jpmorgan_qa_section.csv")
qa_data.head()

,speaker,job_title,marker,utterance,filename,financial_quarter,call_date
0,"Jim Mitchell, Analyst, Seaport Global Securiti...","Analyst, Seaport Global Securities LLC",Q,"Thanks. Good morning. Hey, Jeremy, you talked ...",2q23-earnings-transcript.pdf,2Q23,2023-07-14
1,"Jeremy Barnum, Chief Financial Officer, JPMorg...","Chief Financial Officer, JPMorgan Chase & Co.",A,"Yeah, sure. Thanks, Jim. So, yeah, so when we ...",2q23-earnings-transcript.pdf,2Q23,2023-07-14
2,"Jim Mitchell, Analyst, Seaport Global Securiti...","Analyst, Seaport Global Securities LLC",Q,Okay. But I guess just one follow-up on that. ...,2q23-earnings-transcript.pdf,2Q23,2023-07-14
3,"Jeremy Barnum, Chief Financial Officer, JPMorg...","Chief Financial Officer, JPMorgan Chase & Co.",A,I wouldn't necessarily assume that the evoluti...,2q23-earnings-transcript.pdf,2Q23,2023-07-14
4,"Jim Mitchell, Analyst, Seaport Global Securiti...","Analyst, Seaport Global Securities LLC",Q,"Okay. Great. Thanks. 5 Operator: Next, we'll g...",2q23-earnings-transcript.pdf,2Q23,2023-07-14


In [411]:
#ascend df by financial_quarter
quarter_order = ["1Q23", "2Q23","3Q23","4Q23","1Q24","2Q24","3Q24","4Q24"]
qa_data["financial_quarter"] = pd.Categorical(qa_data["financial_quarter"], categories=quarter_order, ordered=True)
qa_data = qa_data.sort_values("financial_quarter", kind="mergesort")

In [412]:
#preprocessing data
preprocessor(qa_data, "utterance", "question_tokenised_data", "question_cleaned")
preprocessor(qa_data,"utterance","answer_tokenised_data","answer_cleaned")

In [413]:
#remove operater
#qa_data = qa_data.loc[qa_data["speaker"]!="Operator"]

#remove less than 20 words
qa_data["count"] = qa_data["question_tokenised_data"].apply(lambda x: len(x))
qa_data = qa_data.loc[qa_data["count"]>20]
qa_data.head()

,speaker,job_title,marker,utterance,filename,financial_quarter,call_date,question_tokenised_data,question_cleaned,answer_tokenised_data,answer_cleaned,count
216,"Steven Chubak, Analyst, Wolfe Research LLC","Analyst, Wolfe Research LLC",Q,"So, Jamie, I was actually hoping to get your p...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[jamie, actually, hoping, get, perspective, se...",[so jamie actually hoping get perspective see ...,"[jamie, actually, hoping, get, perspective, se...",[so jamie actually hoping get perspective see ...,54
217,"Jamie Dimon, Chairman & Chief Executive Office...","Chairman & Chief Executive Officer, JPMorgan C...",A,"Well, I think you were already kind of complet...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[well, think, already, kind, complete, answeri...",[well think already kind complete answering qu...,"[well, think, already, kind, complete, answeri...",[well think already kind complete answering qu...,84
221,"Jamie Dimon, Chairman & Chief Executive Office...","Chairman & Chief Executive Officer, JPMorgan C...",A,"Well, we've told you that we're kind of pencil...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[well, ve, told, re, kind, penciling, billion,...",[well weve told were kind penciling $12 billio...,"[well, ve, told, re, kind, penciling, billion,...",[well weve told were kind penciling $12 billio...,30
224,"Ken Usdin, Analyst, Jefferies LLC","Analyst, Jefferies LLC",Q,"Hey, thanks. Good morning. Hey, Jeremy, I was ...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[hey, thanks, good, morning, hey, jeremy, wond...",[hey thanks good morning hey jeremy wondering ...,"[hey, thanks, good, morning, hey, jeremy, wond...",[hey thanks good morning hey jeremy wondering ...,35
225,"Jeremy Barnum, Chief Financial Officer, JPMorg...","Chief Financial Officer, JPMorgan Chase & Co.",A,"Yeah, sure. So let me just summarize the drive...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[yeah, sure, let, summarize, driver, change, o...",[yeah sure let summarize drivers change outloo...,"[yeah, sure, let, summarize, driver, change, o...",[yeah sure let summarize drivers change outloo...,111


In [414]:
#reset index
qa_data.reset_index(drop=True, inplace=True)
qa_data.head()

,speaker,job_title,marker,utterance,filename,financial_quarter,call_date,question_tokenised_data,question_cleaned,answer_tokenised_data,answer_cleaned,count
0,"Steven Chubak, Analyst, Wolfe Research LLC","Analyst, Wolfe Research LLC",Q,"So, Jamie, I was actually hoping to get your p...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[jamie, actually, hoping, get, perspective, se...",[so jamie actually hoping get perspective see ...,"[jamie, actually, hoping, get, perspective, se...",[so jamie actually hoping get perspective see ...,54
1,"Jamie Dimon, Chairman & Chief Executive Office...","Chairman & Chief Executive Officer, JPMorgan C...",A,"Well, I think you were already kind of complet...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[well, think, already, kind, complete, answeri...",[well think already kind complete answering qu...,"[well, think, already, kind, complete, answeri...",[well think already kind complete answering qu...,84
2,"Jamie Dimon, Chairman & Chief Executive Office...","Chairman & Chief Executive Officer, JPMorgan C...",A,"Well, we've told you that we're kind of pencil...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[well, ve, told, re, kind, penciling, billion,...",[well weve told were kind penciling $12 billio...,"[well, ve, told, re, kind, penciling, billion,...",[well weve told were kind penciling $12 billio...,30
3,"Ken Usdin, Analyst, Jefferies LLC","Analyst, Jefferies LLC",Q,"Hey, thanks. Good morning. Hey, Jeremy, I was ...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[hey, thanks, good, morning, hey, jeremy, wond...",[hey thanks good morning hey jeremy wondering ...,"[hey, thanks, good, morning, hey, jeremy, wond...",[hey thanks good morning hey jeremy wondering ...,35
4,"Jeremy Barnum, Chief Financial Officer, JPMorg...","Chief Financial Officer, JPMorgan Chase & Co.",A,"Yeah, sure. So let me just summarize the drive...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[yeah, sure, let, summarize, driver, change, o...",[yeah sure let summarize drivers change outloo...,"[yeah, sure, let, summarize, driver, change, o...",[yeah sure let summarize drivers change outloo...,111


In [415]:
#Create question number column
create_ques_num_column(qa_data,"question_number_inline","marker")

<ipython-input-403-53017643a2e6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[new_col]=None


In [416]:
qa_data.head()

,speaker,job_title,marker,utterance,filename,financial_quarter,call_date,question_tokenised_data,question_cleaned,answer_tokenised_data,answer_cleaned,count,question_number_inline
0,"Steven Chubak, Analyst, Wolfe Research LLC","Analyst, Wolfe Research LLC",Q,"So, Jamie, I was actually hoping to get your p...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[jamie, actually, hoping, get, perspective, se...",[so jamie actually hoping get perspective see ...,"[jamie, actually, hoping, get, perspective, se...",[so jamie actually hoping get perspective see ...,54,0
1,"Jamie Dimon, Chairman & Chief Executive Office...","Chairman & Chief Executive Officer, JPMorgan C...",A,"Well, I think you were already kind of complet...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[well, think, already, kind, complete, answeri...",[well think already kind complete answering qu...,"[well, think, already, kind, complete, answeri...",[well think already kind complete answering qu...,84,None
2,"Jamie Dimon, Chairman & Chief Executive Office...","Chairman & Chief Executive Officer, JPMorgan C...",A,"Well, we've told you that we're kind of pencil...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[well, ve, told, re, kind, penciling, billion,...",[well weve told were kind penciling $12 billio...,"[well, ve, told, re, kind, penciling, billion,...",[well weve told were kind penciling $12 billio...,30,None
3,"Ken Usdin, Analyst, Jefferies LLC","Analyst, Jefferies LLC",Q,"Hey, thanks. Good morning. Hey, Jeremy, I was ...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[hey, thanks, good, morning, hey, jeremy, wond...",[hey thanks good morning hey jeremy wondering ...,"[hey, thanks, good, morning, hey, jeremy, wond...",[hey thanks good morning hey jeremy wondering ...,35,1
4,"Jeremy Barnum, Chief Financial Officer, JPMorg...","Chief Financial Officer, JPMorgan Chase & Co.",A,"Yeah, sure. So let me just summarize the drive...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[yeah, sure, let, summarize, driver, change, o...",[yeah sure let summarize drivers change outloo...,"[yeah, sure, let, summarize, driver, change, o...",[yeah sure let summarize drivers change outloo...,111,None


In [417]:
#adding new columns
qa_data["analyst"]=None
qa_data["analyst_title"]=None
qa_data["metadata_question"]=None
qa_data["question_num"]=None

#

for i in qa_data.index:
    name_list=[]
    title_list=[]
    question_list=[]
    quest_num_list=[]
    #If the value in the "marker" column at row i is "Q", then set the values of "analyst", "analyst_title", "metadata_question", and "question_num" columns at row i to "x".
    if qa_data.loc[i,"marker"]=="Q":
      qa_data.at[i,"analyst"]="x"
      qa_data.at[i,"analyst_title"]="x"
      qa_data.at[i,"metadata_question"]="x"
      qa_data.at[i,"question_num"]="x"
    elif find_last_a(qa_data,"marker",i) >=0 and qa_data.loc[i-1,"marker"]=="Q":
      name_list.append(qa_data["speaker"][i-1])
      title_list.append(qa_data["job_title"][i-1])
      question_list.append(qa_data["utterance"][i-1])
      quest_num_list.append(str(qa_data["question_number_inline"][i-1]))
    elif find_last_a(qa_data,"marker",i) >=0 and qa_data.loc[i-1,"marker"]=="A":
      qa_data.at[i,"analyst"] = qa_data.at[i-1,"analyst"]
      qa_data.at[i, "analyst_title"] = qa_data.at[i-1,"analyst_title"]
      qa_data.at[i,"metadata_question"]=qa_data.at[i-1, "metadata_question"]
      qa_data.at[i,"question_num"] = qa_data.at[i-1, "question_num"]

    #Initialize lists and populate them with "speaker", "job_title", "utterance", and "question_number_inline" values from rows between last_a and i in qa_data.
    else:
      last_a = find_row(qa_data,"marker", i)+ 1
      for j in range(last_a, i):
        name_list.append(qa_data["speaker"][j])
        title_list.append(qa_data["job_title"][j])
        question_list.append(qa_data["utterance"][j])
        quest_num_list.append(str(qa_data["question_number_inline"][j]))
 # Assign the lists name_list, title_list, question_list, and quest_num_list to the "analyst", "analyst_title", "metadata_question", and "question_num" columns at row i in qa_data, respectively.
    qa_data.at[i,"analyst"]=name_list
    qa_data.at[i,"analyst_title"]=title_list
    qa_data.at[i,"metadata_question"]=question_list
    qa_data.at[i,"question_num"]=quest_num_list

In [418]:
qa_data.head()

,speaker,job_title,marker,utterance,filename,financial_quarter,call_date,question_tokenised_data,question_cleaned,answer_tokenised_data,answer_cleaned,count,question_number_inline,analyst,analyst_title,metadata_question,question_num
0,"Steven Chubak, Analyst, Wolfe Research LLC","Analyst, Wolfe Research LLC",Q,"So, Jamie, I was actually hoping to get your p...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[jamie, actually, hoping, get, perspective, se...",[so jamie actually hoping get perspective see ...,"[jamie, actually, hoping, get, perspective, se...",[so jamie actually hoping get perspective see ...,54,0,[],[],[],[]
1,"Jamie Dimon, Chairman & Chief Executive Office...","Chairman & Chief Executive Officer, JPMorgan C...",A,"Well, I think you were already kind of complet...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[well, think, already, kind, complete, answeri...",[well think already kind complete answering qu...,"[well, think, already, kind, complete, answeri...",[well think already kind complete answering qu...,84,None,"[Steven Chubak, Analyst, Wolfe Research LLC]","[Analyst, Wolfe Research LLC]","[So, Jamie, I was actually hoping to get your ...",[0]
2,"Jamie Dimon, Chairman & Chief Executive Office...","Chairman & Chief Executive Officer, JPMorgan C...",A,"Well, we've told you that we're kind of pencil...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[well, ve, told, re, kind, penciling, billion,...",[well weve told were kind penciling $12 billio...,"[well, ve, told, re, kind, penciling, billion,...",[well weve told were kind penciling $12 billio...,30,None,[],[],[],[]
3,"Ken Usdin, Analyst, Jefferies LLC","Analyst, Jefferies LLC",Q,"Hey, thanks. Good morning. Hey, Jeremy, I was ...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[hey, thanks, good, morning, hey, jeremy, wond...",[hey thanks good morning hey jeremy wondering ...,"[hey, thanks, good, morning, hey, jeremy, wond...",[hey thanks good morning hey jeremy wondering ...,35,1,[],[],[],[]
4,"Jeremy Barnum, Chief Financial Officer, JPMorg...","Chief Financial Officer, JPMorgan Chase & Co.",A,"Yeah, sure. So let me just summarize the drive...",1q23-earnings-transcript.pdf,1Q23,2023-04-14,"[yeah, sure, let, summarize, driver, change, o...",[yeah sure let summarize drivers change outloo...,"[yeah, sure, let, summarize, driver, change, o...",[yeah sure let summarize drivers change outloo...,111,None,"[Ken Usdin, Analyst, Jefferies LLC]","[Analyst, Jefferies LLC]","[Hey, thanks. Good morning. Hey, Jeremy, I was...",[1]


In [419]:
for i in range(len(qa_data)):
  #if the first row is answer, then return all analyst columns as x, if the row is answer and there is not analyst, then popurate questioner analyst name and other data, otherwise pass the function
  if i ==0 and qa_data["marker"][i]=="A":
    qa_data.at[i,"analyst"] ="x"
    qa_data.at[i,"analyst_title"] ="x"
    qa_data.at[i,"metadata_question"] ="x"
    qa_data.at[i,"question_num"]="x"
  elif qa_data["marker"][i]=="A"and qa_data["analyst"][i]==[]:
    a = find_row_empty(qa_data,"marker","analyst",i)
    qa_data.at[i,"analyst"] = qa_data.loc[a,"analyst"]
    qa_data.at[i,"analyst_title"] = qa_data.loc[a,"analyst_title"]
    qa_data.at[i,"metadata_question"] = qa_data.loc[a,"metadata_question"]
    qa_data.at[i,"question_num"] =qa_data.loc[a,"question_num"]
  else:
    continue


In [420]:
#rename column
qa_data.rename(columns={"utterance":"metadata"},inplace=True)


In [421]:
#reorganise column
qa_data=qa_data[["filename","financial_quarter","call_date","speaker","marker","question_num","job_title","metadata", "answer_cleaned","analyst","analyst_title","metadata_question","question_cleaned"]]

In [422]:
# Apply the function to the DataFrame
qa_data['speaker'] = qa_data['speaker'].apply(extract_name)

<ipython-input-422-3b2ec04a312b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_data['speaker'] = qa_data['speaker'].apply(extract_name)


In [423]:
"""
#ascend df by financial_quarter
quarter_order = ["1Q23", "2Q23","3Q23","4Q23","1Q24","2Q24","3Q24","4Q24"]
qa_data["financial_quarter"] = pd.Categorical(qa_data["financial_quarter"], categories=quarter_order, ordered=True)
df_sorted = qa_data.sort_values("financial_quarter", kind="mergesort")
"""

'\n#ascend df by financial_quarter\nquarter_order = ["1Q23", "2Q23","3Q23","4Q23","1Q24","2Q24","3Q24","4Q24"]\nqa_data["financial_quarter"] = pd.Categorical(qa_data["financial_quarter"], categories=quarter_order, ordered=True)\ndf_sorted = qa_data.sort_values("financial_quarter", kind="mergesort")\n'

In [424]:
qa_data.head(20)

,filename,financial_quarter,call_date,speaker,marker,question_num,job_title,metadata,answer_cleaned,analyst,analyst_title,metadata_question,question_cleaned
0,1q23-earnings-transcript.pdf,1Q23,2023-04-14,Steven Chubak,Q,[],"Analyst, Wolfe Research LLC","So, Jamie, I was actually hoping to get your p...",[so jamie actually hoping get perspective see ...,[],[],[],[so jamie actually hoping get perspective see ...
1,1q23-earnings-transcript.pdf,1Q23,2023-04-14,Jamie Dimon,A,[0],"Chairman & Chief Executive Officer, JPMorgan C...","Well, I think you were already kind of complet...",[well think already kind complete answering qu...,"[Steven Chubak, Analyst, Wolfe Research LLC]","[Analyst, Wolfe Research LLC]","[So, Jamie, I was actually hoping to get your ...",[well think already kind complete answering qu...
2,1q23-earnings-transcript.pdf,1Q23,2023-04-14,Jamie Dimon,A,[0],"Chairman & Chief Executive Officer, JPMorgan C...","Well, we've told you that we're kind of pencil...",[well weve told were kind penciling $12 billio...,"[Steven Chubak, Analyst, Wolfe Research LLC]","[Analyst, Wolfe Research LLC]","[So, Jamie, I was actually hoping to get your ...",[well weve told were kind penciling $12 billio...
3,1q23-earnings-transcript.pdf,1Q23,2023-04-14,Ken Usdin,Q,[],"Analyst, Jefferies LLC","Hey, thanks. Good morning. Hey, Jeremy, I was ...",[hey thanks good morning hey jeremy wondering ...,[],[],[],[hey thanks good morning hey jeremy wondering ...
4,1q23-earnings-transcript.pdf,1Q23,2023-04-14,Jeremy Barnum,A,[1],"Chief Financial Officer, JPMorgan Chase & Co.","Yeah, sure. So let me just summarize the drive...",[yeah sure let summarize drivers change outloo...,"[Ken Usdin, Analyst, Jefferies LLC]","[Analyst, Jefferies LLC]","[Hey, thanks. Good morning. Hey, Jeremy, I was...",[yeah sure let summarize drivers change outloo...
5,1q23-earnings-transcript.pdf,1Q23,2023-04-14,Ken Usdin,Q,[],"Analyst, Jefferies LLC","Yeah, and as a follow-up on the point about ra...",[yeah follow-up point rate expectations coming...,[],[],[],[yeah follow-up point rate expectations coming...
6,1q23-earnings-transcript.pdf,1Q23,2023-04-14,Jamie Dimon,A,[2],"Chairman & Chief Executive Officer, JPMorgan C...","Well first of all, I don't quite believe it. S...",[well first all quite believe it fed rate curv...,"[Ken Usdin, Analyst, Jefferies LLC]","[Analyst, Jefferies LLC]","[Yeah, and as a follow-up on the point about r...",[well first all quite believe it fed rate curv...
7,1q23-earnings-transcript.pdf,1Q23,2023-04-14,John McDonald,Q,[],"Analyst, Autonomous Research","Hi, thanks. Jeremy, wanted to follow up again ...",[hi thanks jeremy wanted follow drivers nii re...,[],[],[],[hi thanks jeremy wanted follow drivers nii re...
8,1q23-earnings-transcript.pdf,1Q23,2023-04-14,Jeremy Barnum,A,[3],"Chief Financial Officer, JPMorgan Chase & Co.","Yeah. John, it's a really good question, and w...",[yeah john really good question weve obviously...,"[John McDonald, Analyst, Autonomous Research]","[Analyst, Autonomous Research]","[Hi, thanks. Jeremy, wanted to follow up again...",[yeah john really good question weve obviously...
9,1q23-earnings-transcript.pdf,1Q23,2023-04-14,John McDonald,Q,[],"Analyst, Autonomous Research",Okay. And then I wanted to ask Jamie – there's...,[okay wanted ask jamie – theres narrative indu...,[],[],[],[okay wanted ask jamie – theres narrative indu...


JP morgan management discussion

In [ ]:
%ls

In [ ]:
#defining santader dataframe
jpmorgan_body_df=pd.read_csv("jpmorgan_management_discussion.csv")
jpmorgan_body_df.head()

In [ ]:
#preprocess data
preprocessor(jpmorgan_body_df, "chunk_text", "tokenized_data","cleaned_data")

In [ ]:
jpmorgan_body_df.head()

UBS qna section

In [ ]:
%ls

In [ ]:
#define ubs q&a data
ubs_qna_df=pd.read_csv("ubs_qna_section.csv")

In [ ]:
#preprocessing ubs Q&A data
preprocessor(ubs_qna_df, "utterance", "tokenized_data","cleaned_data")

In [ ]:
ubs_qna_df.head()

UBS management discussion

In [ ]:
%ls

In [ ]:
#defining ubs management discussion
ubs_manag_df=pd.read_csv("ubs_management_discussion.csv")
ubs_manag_df.head()

In [ ]:
#preprocessing ubs management discussion
preprocessor(ubs_manag_df,"utterance", "tokenized_data","cleaned_data")
ubs_manag_df.head()

# **Export the output as a csv file**

JP morgan QA section

In [425]:
#export preprocessed data
preprocessed_qa_csv_path1 = "/content/drive/MyDrive/bank_of_england/data/preprocessed_data/jpmorgan_qna_df_preprocessed_ver7.csv"
qa_data.to_csv(preprocessed_qa_csv_path1, index=False)

JP morgan management discussion

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path2 = "/content/sample_data/jpmorgan_management_df_preprocessed.csv"
jpmorgan_body_df.to_csv(preprocessed_qa_csv_path2, index=False)

UBS QA section

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path3 = "/content/sample_data/ubs_qa_df_preprocessed.csv"
ubs_qna_df.to_csv(preprocessed_qa_csv_path3, index=False)

UBS management discussion

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path4 = "/content/sample_data/ubs_management_df_preprocessed.csv"
ubs_manag_df.to_csv(preprocessed_qa_csv_path4, index=False)